# Executive Summary: Data Integration Assessment

## Key Findings

**✅ What's Working:**
- Team name unification across sources (relocations tracked correctly)
- Attendance data successfully integrated (historical 2000-2019 + current 2020-2024)
- Database infrastructure solid (1.5 MB, all tables/views present)

**❌ Critical Gap Identified:**
- **Performance metrics isolated in separate tables** - wins, losses, playoff status, rankings exist in `kaggle_standings` but are not integrated with attendance data
- **View design is source-centric, not analysis-centric** - current views split by data origin rather than business questions
- **Analyst workload increased** - requires manual 3-4 table joins for basic questions like "Does winning drive attendance?"

## Recommendation
Create an integrated analysis view (`v_team_performance_attendance`) that joins attendance + performance + team info, enabling direct business queries without complex joins.

---

**Supporting Analysis Below** ⬇️

In [ ]:
## 1: Imports & Setup

# Standard library
import os
import sys
from pathlib import Path
import sqlite3

# Third-Party
import pandas as pd

# Local / Path Config
# (Assumes: notebooks/01_exploratory_analysis.ipynb)
PROJECT_ROOT = Path('../').resolve()
sys.path.insert(0, str(PROJECT_ROOT))
from src.utils.config import DB_PATH, DB_TABLES, VIEW_FILES
db_relative = DB_PATH.relative_to(PROJECT_ROOT)  # Extract relative path from project root for cleaner display

print("Imports and setup successful...")


Imports and setup successful...


In [21]:
## 2: Check if the database file exists and has size

# File Exists
if not os.path.exists(DB_PATH):
    raise FileNotFoundError(f"❌ Database not found: {DB_PATH}")

# Size >= 1KB
db_size = os.path.getsize(DB_PATH)
if db_size < 1000:  # Less than 1KB = probably empty
    raise ValueError(f"❌ Database appears empty ({db_size} bytes). Run load_to_database.py first.")

print(f"Database file exists at {DB_PATH}.")
print(f"size = ({db_size} bytes).")

Database file exists at C:\Users\Barac0da\source\repos\GameDay_Analytics\data\processed\nfl_attendance.db.
size = (1499136 bytes).


In [11]:
## 3: Connect to database
conn = sqlite3.connect(DB_PATH)

In [12]:
## 4: Check expected tables and views exist
# (SQLite's version of Oracle's USER_TABLES / USER_VIEWS)
cursor = conn.cursor()

# Expected Tables
found_tables = cursor.execute(
    "SELECT name FROM sqlite_master WHERE type='table'"
).fetchall()
found_table_names = [t[0] for t in found_tables]

missing = [t for t in DB_TABLES if t not in found_table_names]
if missing:
    raise ValueError(f"❌ Missing required tables: {missing}")

# Expected Views
DB_VIEWS = [v.replace('.sql', '') for v in VIEW_FILES]  # strip .sql extension
found_views = cursor.execute(
    "SELECT name FROM sqlite_master WHERE type='view'"
).fetchall()
found_view_names = [v[0] for v in found_views]
missing_views = [v for v in DB_VIEWS if v not in found_view_names]
if missing_views:
    raise ValueError(f"❌ Missing required views: {missing_views}")

# Summary
print(f"✅ Connected to valid database: {DB_PATH} ({db_size:,} bytes)")
print(f"✅ Found {len(found_table_names)} out of {len(DB_TABLES)} expected tables")
print(f"✅ Found {len(found_view_names)} out of {len(DB_VIEWS)} expected tables")

✅ Connected to valid database: C:\Users\Barac0da\source\repos\GameDay_Analytics\data\processed\nfl_attendance.db (1,499,136 bytes)
✅ Found 6 out of 6 expected tables
✅ Found 3 out of 3 expected tables


In [13]:
## 5. "What do we have" Data Viz

# Show schema with row counts
schema_info = []
for table in found_table_names:
    count = cursor.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    schema_info.append({'name': table, 'type': 'table', 'rows': count})

for view in found_view_names:
    count = cursor.execute(f"SELECT COUNT(*) FROM {view}").fetchone()[0]
    schema_info.append({'name': view, 'type': 'view', 'rows': count})

schema_df = pd.DataFrame(schema_info)

# Split into tables vs views
tables_df = schema_df[schema_df['type'] == 'table'][['name', 'rows']].reset_index(drop=True)
views_df = schema_df[schema_df['type'] == 'view'][['name', 'rows']].reset_index(drop=True)

# Display side-by-side with header (notebooks stack vertically - use HTML)
from IPython.display import display, HTML

# Extract relative path from project root for cleaner display
db_relative = DB_PATH.relative_to(PROJECT_ROOT)

display(HTML(
    f"<h3>✅ Connected! </h3>"
    f"<p style='color: #666; margin-bottom: 20px;'>{db_relative} ({db_size:,} bytes)</p>"
    "<div style='display: flex; gap: 40px;'>"
    f"<div><h4>Tables ({len(tables_df)})</h4>{tables_df.to_html(index=False)}</div>"
    f"<div><h4>Views ({len(views_df)})</h4>{views_df.to_html(index=False)}</div>"
    "</div>"
))

name,rows
kaggle_attendance,10846
kaggle_games,5324
kaggle_standings,638
espn_teams,32
espn_games,1452
team_reference,37
name,rows
v_teams_unified,37
v_attendance_historical,10846
v_attendance_current,1292


In [14]:
## 6: First Peek at Views (Our Working Data)

from IPython.display import display, HTML

# Config
SET_MAX_VIEW_PREVIEW = 5

# Loop through views (limit to max)
view_html = ""
for view_name in found_view_names[:SET_MAX_VIEW_PREVIEW]:
    # Get row count
    row_count = cursor.execute(f"SELECT COUNT(*) FROM {view_name}").fetchone()[0]
    
    # Get head
    df_head = pd.read_sql_query(f"SELECT * FROM {view_name} LIMIT 5", conn)
    
    # Column list
    col_list = "<br>".join(df_head.columns.tolist())
    
    view_html += f"""
    <h2>{view_name} <span style='color: #666; font-weight: normal;'>({row_count:,} rows)</span></h2>
    <hr style='border: 1px solid #ddd; margin: 5px 0 15px 0;'>
    <div style='display: flex; gap: 30px; margin-bottom: 40px;'>
        <div style='min-width: 200px;'>
            <strong>Columns ({len(df_head.columns)})</strong><br>
            <span style='color: #666; font-size: 0.9em;'>{col_list}</span>
        </div>
        <div style='flex-grow: 1;'>
            <p style='color: #666; margin: 0 0 10px 0;'><strong>First 5 rows:</strong></p>
            {df_head.to_html(index=False)}
        </div>
    </div>
    """

display(HTML(view_html))

In [15]:
## What performance data do we actually have?
standings = pd.read_sql_query("SELECT * FROM kaggle_standings LIMIT 5", conn)
print("kaggle_standings columns:", standings.columns.tolist())
standings.head()

kaggle_standings columns: ['team', 'team_name', 'year', 'wins', 'loss', 'points_for', 'points_against', 'points_differential', 'margin_of_victory', 'strength_of_schedule', 'simple_rating', 'offensive_ranking', 'defensive_ranking', 'playoffs', 'sb_winner']


,team,team_name,year,wins,loss,points_for,points_against,points_differential,margin_of_victory,strength_of_schedule,simple_rating,offensive_ranking,defensive_ranking,playoffs,sb_winner
0,Miami,Dolphins,2000,11,5,323,226,97,6.1,1.0,7.1,0.0,7.1,Playoffs,No Superbowl
1,Indianapolis,Colts,2000,10,6,429,326,103,6.4,1.5,7.9,7.1,0.8,Playoffs,No Superbowl
2,New York,Jets,2000,9,7,321,321,0,0.0,3.5,3.5,1.4,2.2,No Playoffs,No Superbowl
3,Buffalo,Bills,2000,8,8,315,350,-35,-2.2,2.2,0.0,0.5,-0.5,No Playoffs,No Superbowl
4,New England,Patriots,2000,5,11,276,338,-62,-3.9,1.4,-2.5,-2.7,0.2,No Playoffs,No Superbowl


## Findings: Data Integration Gaps

While exploring the database, I discovered:

**✅ What's Working:**
- Team name unification across sources (relocations tracked)
- Attendance data integrated (historical + current)

**❌ What's Missing:**
- Performance metrics (wins/losses/rankings) are in separate `kaggle_standings` table
- Analyst must manually join 3+ tables to answer core business questions
- View names are source-oriented (`v_attendance_historical`) rather than analysis-oriented

**Recommendation for Linda:**
Create an integrated analysis view that joins:
- Attendance (from existing views)
- Performance (from `kaggle_standings`)
- Team info (from `v_teams_unified`)

This would enable direct queries like "How does win percentage correlate with attendance?"

In [16]:
## Proposed View Schema (for Linda)

# What Ronald needs: attendance + performance in one queryable view
proposed_view = """
CREATE VIEW v_team_performance_attendance AS
SELECT 
    -- Team & Time
    t.team_name,
    t.location,
    s.year,
    
    -- Performance Metrics
    s.wins,
    s.loss,
    s.playoffs,
    s.sb_winner,
    s.points_for,
    s.points_against,
    
    -- Attendance Metrics
    a.total_attendance,
    a.home_attendance,
    a.avg_attendance_per_game
    
FROM v_teams_unified t
JOIN kaggle_standings s ON t.kaggle_team_name = s.team_name AND t.active_years ...
JOIN v_attendance_historical a ON ...
"""

print("Proposed view structure for business analysis:")
print(proposed_view)

Proposed view structure for business analysis:

CREATE VIEW v_team_performance_attendance AS
SELECT 
    -- Team & Time
    t.team_name,
    t.location,
    s.year,

    -- Performance Metrics
    s.wins,
    s.loss,
    s.playoffs,
    s.sb_winner,
    s.points_for,
    s.points_against,

    -- Attendance Metrics
    a.total_attendance,
    a.home_attendance,
    a.avg_attendance_per_game

FROM v_teams_unified t
JOIN kaggle_standings s ON t.kaggle_team_name = s.team_name AND t.active_years ...
JOIN v_attendance_historical a ON ...

